In [1]:
from utils import *
import pandas as pd
from time import time
from ptflops import get_model_complexity_info
import matplotlib.pyplot as plt
from tqdm import tqdm
from PAD import PAD
import os
import matplotlib.pyplot as plt
import pandas as pd
from time import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class Args:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

def train(net, trainloader, criterion, optimizer,device=device, epochs=20):
    net.train()
    net.to(device)
    #create args.device 
    args = Args()
    args.device = device
    progress_bar = tqdm(range(epochs), desc=f'Training - No data available', leave=True)

    for i in progress_bar:
        loss_avg, acc_avg, num_exp = 0, 0, 0
        for i_batch, datum in enumerate(trainloader):
            img = datum[0].float().to(args.device)
            lab = datum[1].long().to(args.device)
            n_b = lab.shape[0]
            output = net(img)[1]
            loss = criterion(output, lab)
            acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))
            loss_avg += loss.item()*n_b
            acc_avg += acc
            num_exp += n_b

            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_avg /= num_exp
        acc_avg /= num_exp
        progress_bar.set_description(f'Training - Loss: {loss_avg:.4f} - Accuracy: {acc_avg:.4f}')
    return net

def test(net, testloader, criterion, optimizer,device=device, return_acc=False):
    net.eval()
    net.to(device)
    args = Args()
    args.device = device
    loss_avg, acc_avg, num_exp = 0, 0, 0
    for i_batch, datum in enumerate(testloader):
        img = datum[0].float().to(args.device)
        lab = datum[1].long().to(args.device)
        n_b = lab.shape[0]

        output = net(img)[1]
        loss = criterion(output, lab)
        acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))
        loss_avg += loss.item()*n_b
        acc_avg += acc
        num_exp += n_b


    loss_avg /= num_exp
    acc_avg /= num_exp
    
    if return_acc:
        return acc_avg
    else:
        print(f'Accuracy of the network on the test images: {acc_avg*100}%')

def count_flops(net, channel, im_size):
    param = (channel, im_size[0], im_size[1])
    flops, params = get_model_complexity_info(net, param, as_strings=True, print_per_layer_stat=False)
    print(f'FLOPs: {flops}')
    print(f'Params: {params}')

### Distillation

#### MNIST

In [ ]:
#MNIST Dataset
data_path = "./Project A/PAD"

#create a folder to save the synthetic dataset
if not os.path.exists(os.path.join(data_path, 'synthetic_dataset_MNIST_gaussian')):
    os.makedirs(os.path.join(data_path, 'synthetic_dataset_MNIST_gaussian'))

#load the dataset
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset("MNIST", data_path)

#define the model
net = 'ConvNetD3'

Distillator = PAD(device, net, channel, num_classes, im_size, dst_train, 'synthetic_dataset_MNIST_gaussian', batch_size = 64, M=2,
                 K=100, T=10, eta_S=0.1, zeta_S=1, eta_theta=0.01, zeta_theta=50, alpha=0.01, minibatches_size=64)

mean_gaussian = 0
std_gaussian = 1
#Distillator.initialize_synthetic_dataset_from_gaussian_noise(mean_gaussian,std_gaussian)
#Distillator.initialize_synthetic_dataset_from_real()

condensed_dataset = Distillator.train("Gaussian", mean_gaussian, std_gaussian)


